#<h1> IG calculation </h1>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
FOLDERNAME = 'lemm_stop/part1'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Generating Spam email count and ham email count

In [3]:
dir_list = ['/content/drive/My Drive/lemm_stop/part1', '/content/drive/My Drive/lemm_stop/part2', '/content/drive/My Drive/lemm_stop/part3', '/content/drive/My Drive/lemm_stop/part4',\
            '/content/drive/My Drive/lemm_stop/part5', '/content/drive/My Drive/lemm_stop/part6', '/content/drive/My Drive/lemm_stop/part7', '/content/drive/My Drive/lemm_stop/part8',\
            '/content/drive/My Drive/lemm_stop/part9']         

spam_count = 0
ham_count = 0
import os
for directory_in_str in dir_list:
    directory = os.fsencode(directory_in_str)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".txt"):
            if 'sp' in filename:
                spam_count+=1
            else:
                ham_count+=1               
        else:
            continue
print(spam_count, ham_count)            

432 2170


In [4]:
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]

In [5]:
import os
import re
import string
from pathlib import Path
import nltk
nltk.download('words')
wd = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


<h2>Processing train data</h2>
Processing includes converting all characters to lower case, removing numbers, removing all punctuations, removing new line and tab spaces, stripping spaces at the beginning and at the end if the email, removing the word 'subject'. 
After this is done all the email is grouped into either spam or legit categories. The same process is applied on the test set as well.

In [6]:
dir_list = ['/content/drive/My Drive/lemm_stop/part1/', '/content/drive/My Drive/lemm_stop/part2/', '/content/drive/My Drive/lemm_stop/part3/', '/content/drive/My Drive/lemm_stop/part4/',\
            '/content/drive/My Drive/lemm_stop/part5/', '/content/drive/My Drive/lemm_stop/part6/', '/content/drive/My Drive/lemm_stop/part7/', '/content/drive/My Drive/lemm_stop/part8/',\
            '/content/drive/My Drive/lemm_stop/part9/']   

import os
import re
import string
from pathlib import Path
import nltk
nltk.download('words')
wd = set(nltk.corpus.words.words())
cts = 0
wordsets = []
spam_docs = []
legit_docs = []

processed_spam_docs = []
processed_legit_docs = []

for directory_in_str in dir_list:
    directory = os.fsencode(directory_in_str)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".txt"):
            data = Path(directory_in_str + filename).read_text()
            data = data.lower()
            data = re.sub(r'\d+', '', data)
            data = data.translate(str.maketrans('','', string.punctuation))
            data = data.replace('\n','')
            data = data.replace('\t','')
            data = data.strip()
            data = [w for w in data.split() if len(w)>1]
            data.remove('subject')
            data = ' '.join(remove_values_from_list(data, 'np'))
            result = []
            for w in nltk.wordpunct_tokenize(data):
                if w.lower() in wd:
                    result.append(w)
            result = ' '.join(result)

            if 'sp' in filename:
                processed_spam_docs.append(result.split(' '))
                spam_docs.append(frozenset(result.split(' ')))
            else:
                processed_legit_docs.append(result.split(' '))
                legit_docs.append(frozenset(result.split(' ')))
        else:
            continue
          

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


<h2>Processing test data</h2>

In [7]:
dir_list = ['/content/drive/My Drive/lemm_stop/part9/']   

import os
import re
import string
from pathlib import Path
import nltk
nltk.download('words')
wd = set(nltk.corpus.words.words())

spam_docs_test = []
legit_docs_test = []
processed_spam_docs_test = []
processed_legit_docs_test = []

for directory_in_str in dir_list:
    directory = os.fsencode(directory_in_str)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".txt"):
            data = Path(directory_in_str + filename).read_text()
            data = data.lower()
            data = re.sub(r'\d+', '', data)
            data = data.translate(str.maketrans('','', string.punctuation))
            data = data.replace('\n','')
            data = data.replace('\t','')
            data = data.strip()
            data = [w for w in data.split() if len(w)>1]
            data.remove('subject')
            data = ' '.join(remove_values_from_list(data, 'np'))
            result = []
            for w in nltk.wordpunct_tokenize(data):
                if w.lower() in wd:
                    result.append(w)
            result = ' '.join(result)

            if 'sp' in filename:
                processed_spam_docs_test.append(result.split(' '))
                spam_docs_test.append(frozenset(result.split(' ')))
            else:
                processed_legit_docs_test.append(result.split(' '))
                legit_docs_test.append(frozenset(result.split(' ')))
        else:
            continue
          

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [8]:
spam_docs_counts = []
legit_docs_counts = []
for word in wd:
    count = sum( 1 for s in spam_docs if word in s )
    if count>0:
        spam_docs_counts.append((word, count))  

for word in wd:
    count = sum( 1 for s in legit_docs if word in s )
    if count>0:
        legit_docs_counts.append((word, count))       

In [9]:
spam_docs_counts_d = {}
legit_docs_counts_d = {}

spam_docs_counts_d = dict(spam_docs_counts)
legit_docs_counts_d = dict(legit_docs_counts)

In [10]:
all_words = set()

for key in spam_docs_counts_d.keys():
    all_words.add(key)

for key in legit_docs_counts_d.keys():
    all_words.add(key)

H(C) value for the data

In [11]:
import numpy as np
p = (spam_count+1)/(spam_count + ham_count +2)
H_of_C = -p*np.log(p) - (1-p)*np.log(1-p) 
print(H_of_C)

0.4499426139156912


IG calculation

In [12]:
IG_dict = {}

for word in all_words:

    try:
        if spam_docs_counts_d[word]:
            sp_word = spam_docs_counts_d[word] + 1
    except KeyError:
        sp_word = 1   

    try:
        if legit_docs_counts_d[word]:
            lg_word = legit_docs_counts_d[word] + 1
    except KeyError:
        lg_word = 1   

    H_C_given_X = (-1)*(((sp_word/(spam_count + ham_count + 2)) * np.log(sp_word/(sp_word + lg_word)))\
                   + ((lg_word/(spam_count + ham_count + 2)) * np.log(lg_word/(sp_word + lg_word)))\
                   + (((spam_count - sp_word)/(spam_count + ham_count + 2)) * np.log((spam_count - sp_word)/(spam_count + ham_count - sp_word - lg_word)))\
                   + (((ham_count - lg_word)/(spam_count + ham_count + 2)) * np.log((ham_count - lg_word)/(spam_count + ham_count - sp_word - lg_word))))


    IG_dict[word] = H_of_C - H_C_given_X



Sorting all IG values in descending order

In [13]:
IG_dict_sorted = {k: v for k, v in sorted(IG_dict.items(), key=lambda item: item[1], reverse=True)}

Extracting the top 10, 100 and 1000 words based on sorted IG and printing the top 10 words along with their IG value.

In [14]:
top_10_IG = []
top_100_IG = []
top_1000_IG = []

top_10_IG_words = []
top_100_IG_words = []
top_1000_IG_words = []

i = 0
print("List of Top 10 words:")
for key in IG_dict_sorted.keys():
    if i>=10:
        break
    top_10_IG.append((key, IG_dict_sorted[key])) 
    top_10_IG_words.append(key) 
    print(key, IG_dict_sorted[key])
    i+=1 

i = 0
for key in IG_dict_sorted.keys():
    if i>=100:
        break
    top_100_IG.append((key, IG_dict_sorted[key]))
    top_100_IG_words.append(key)   
    i+=1   

i = 0
for key in IG_dict_sorted.keys():
    if i>=1000:
        break
    top_1000_IG.append((key, IG_dict_sorted[key])) 
    top_1000_IG_words.append(key)  
    i+=1       

List of Top 10 words:
language 0.1400199328310175
remove 0.11776741926774453
free 0.11516525688714896
linguistic 0.10152229995769935
university 0.09970059497459127
money 0.08321112933282104
click 0.07076502226173614
market 0.06503070551026874
our 0.0628574754117483
business 0.06117324098477023


<h1>Create feature vectors for all documents</h1>

Generating binary feature vectors

In [15]:
def gen_ft_nb(N, spam_or_ham):
    train_data = []
    for doc in spam_or_ham:
        feature_vector = [0]*N
        if N==10:
            wl = top_10_IG_words
        elif N==100:
            wl = top_100_IG_words
        elif N==1000:
            wl = top_1000_IG_words

        for w in wl:
            if w in doc:
                feature_vector[wl.index(w)] = 1
        train_data.append(feature_vector)
    return train_data    


Generating feature vectors for train_data of sizes 10, 100 and 1000

In [16]:
train_data_10_nb = gen_ft_nb(10, processed_legit_docs) + gen_ft_nb(10, processed_spam_docs)

train_data_100_nb = gen_ft_nb(100, processed_legit_docs) + gen_ft_nb(100, processed_spam_docs)

train_data_1000_nb = gen_ft_nb(1000, processed_legit_docs) + gen_ft_nb(1000, processed_spam_docs)

Generating feature vectors for test_data of sizes 10, 100 and 1000

In [17]:
test_data_10_nb = gen_ft_nb(10, processed_legit_docs_test) + gen_ft_nb(10, processed_spam_docs_test)

test_data_100_nb = gen_ft_nb(100, processed_legit_docs_test) + gen_ft_nb(100, processed_spam_docs_test)

test_data_1000_nb = gen_ft_nb(1000, processed_legit_docs_test) + gen_ft_nb(1000, processed_spam_docs_test)

true_labels_test = [0] * 241 + [1] * 48

#<h2>Binomial NB classifier</h2>


In [18]:
P_of_spam = (spam_count+1)/(spam_count + ham_count+2)
P_of_legit = (ham_count+1)/(spam_count + ham_count+2)

def binomial_NB(data, wl):
    P_of_x_given_spam = 1
    P_of_x_given_legit = 1
    for i, w in enumerate(wl):
        try:
            if spam_docs_counts_d[w]:
                sp_word = spam_docs_counts_d[w] + 1           

        except KeyError:
            sp_word = 1    

        try:
            if legit_docs_counts_d[w]:
                lg_word = legit_docs_counts_d[w] + 1

        except KeyError:
            lg_word = 1       

        Pis = sp_word/(spam_count+2)
        Pih = lg_word/(ham_count+2)

        P_of_x_given_spam = P_of_x_given_spam * (np.power(Pis, data[i]) * np.power((1 - Pis),(1 - data[i])))
        P_of_x_given_legit = P_of_x_given_legit * (np.power(Pih, data[i]) * np.power((1 - Pih),(1 - data[i])))

    P_of_x = (P_of_spam * P_of_x_given_spam) + (P_of_legit * P_of_x_given_legit)

    P_of_spam_given_x = (P_of_x_given_spam * P_of_spam) / P_of_x

    P_of_legit_given_x = (P_of_x_given_legit * P_of_legit) / P_of_x


    if P_of_spam_given_x > P_of_legit_given_x:
        return 1
    else:
        return 0   


Prediction on test data with N = 10

In [19]:
pred_labels = []
for fv in test_data_10_nb:
    pred_labels.append(binomial_NB(fv, top_10_IG_words))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

[[238   3]
 [ 10  38]]


In [20]:
def print_spam_precision_recall(cm):
    Spam_precision = (cm[1][1]/(cm[1][1]+cm[1][0]))
    Spam_recall = (cm[1][1]/(cm[1][1]+cm[0][1]))
    Accuracy = (cm[0][0]+cm[1][1])/(289)

    print(f'Spam_precision = {Spam_precision*100}')
    print(f'Spam_recall = {Spam_recall*100}')
    # print(f'Accuracy = {Accuracy*100}')
    return

In [21]:
print_spam_precision_recall(cm)

Spam_precision = 79.16666666666666
Spam_recall = 92.6829268292683



Prediction on test data with N = 100

In [22]:
pred_labels = []
for fv in test_data_100_nb:
    pred_labels.append(binomial_NB(fv, top_100_IG_words))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

[[240   1]
 [ 13  35]]


In [23]:
print_spam_precision_recall(cm)

Spam_precision = 72.91666666666666
Spam_recall = 97.22222222222221



Prediction on test data with N = 1000

In [24]:
pred_labels = []
for fv in test_data_1000_nb:
    pred_labels.append(binomial_NB(fv, top_1000_IG_words))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


[[241   0]
 [ 14  34]]


In [25]:
print_spam_precision_recall(cm)

Spam_precision = 70.83333333333334
Spam_recall = 100.0


#<h2>Multinomial NB with binary features</h2>

Generating term frequencies

In [26]:
import collections
TF_spam = {}
TF_legit = {}

TF_legit = collections.Counter(all_words)
TF_spam = collections.Counter(all_words)

In [27]:
for doc in processed_legit_docs:
    for word in doc:
        TF_legit[word]+=1

for doc in processed_spam_docs:
    for word in doc:
        TF_spam[word]+=1

Generating term frequency feature vectors

In [28]:
import collections
def gen_ft_mnb(N, spam_or_ham):
    train_data = []
    for doc in spam_or_ham:
        doc_dict = collections.Counter(doc)
        feature_vector = [0]*N
        if N==10:
            wl = top_10_IG_words
        elif N==100:
            wl = top_100_IG_words
        elif N==1000:
            wl = top_1000_IG_words

        for w in wl:
            if w in doc_dict:
                feature_vector[wl.index(w)] = doc_dict[w]
        train_data.append(feature_vector)
    return train_data   

Generating feature vectors of size 10, 100 and 1000 with term frequencies for the train dataset

In [29]:
train_data_10_mnb = gen_ft_mnb(10, processed_legit_docs) + gen_ft_mnb(10, processed_spam_docs)

train_data_100_mnb = gen_ft_mnb(100, processed_legit_docs) + gen_ft_mnb(100, processed_spam_docs)

train_data_1000_mnb = gen_ft_mnb(1000, processed_legit_docs) + gen_ft_mnb(1000, processed_spam_docs)

In [30]:
P_of_spam = (spam_count + 1) / (spam_count + ham_count + 2)
P_of_legit = (ham_count + 1) / (spam_count + ham_count + 2)

def multinomial_NB(data, wl):
    P_of_x_given_spam = 1
    P_of_x_given_legit = 1

    M = len(wl)

    TF_legit_sum = 0
    TF_spam_sum = 0

    for word in wl:
        TF_legit_sum += TF_legit[word]
        TF_spam_sum += TF_spam[word]

    for i, w in enumerate(wl):

        Pis = TF_spam[w]/(TF_spam_sum + M)
        Pih = TF_legit[w]/(TF_legit_sum + M)
        
        P_of_x_given_spam = P_of_x_given_spam * (np.power(Pis, data[i]))
        P_of_x_given_legit = P_of_x_given_legit * (np.power(Pih, data[i]))

    P_of_spam_given_x = (P_of_x_given_spam * P_of_spam)
    P_of_legit_given_x = (P_of_x_given_legit * P_of_legit)


    if P_of_spam_given_x / P_of_legit_given_x > 1:
        return 1
    else:
        return 0   


Classification on test set for N=10 

In [31]:
pred_labels = []
for fv in test_data_10_nb:
    pred_labels.append(multinomial_NB(fv, top_10_IG_words))

true_labels_test = [0] * 241 + [1] * 48
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

[[236   5]
 [  9  39]]


In [32]:
print_spam_precision_recall(cm)

Spam_precision = 81.25
Spam_recall = 88.63636363636364


Classification on test set for N=100

In [33]:
pred_labels = []
for fv in test_data_100_nb:
    pred_labels.append(multinomial_NB(fv, top_100_IG_words))

true_labels_test = [0] * 241 + [1] * 48
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

[[239   2]
 [  0  48]]


In [34]:
print_spam_precision_recall(cm)

Spam_precision = 100.0
Spam_recall = 96.0


Classification on test set for N=1000

In [35]:
pred_labels = []
for fv in test_data_1000_nb:
    pred_labels.append(multinomial_NB(fv, top_1000_IG_words))

true_labels_test = [0] * 241 + [1] * 48
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


[[239   2]
 [ 12  36]]


In [36]:
print_spam_precision_recall(cm)

Spam_precision = 75.0
Spam_recall = 94.73684210526315


#<h2>Multinomial NB with TF features</h2>

In [37]:
P_of_spam = (spam_count+1)/(spam_count + ham_count+2)
P_of_legit = (ham_count+1)/(spam_count + ham_count+2)

def multinomial_NB_TF(data, wl):
    P_of_x_given_spam = 1
    P_of_x_given_legit = 1

    M = len(wl)

    TF_legit_sum = 0
    TF_spam_sum = 0

    for word in wl:
        TF_legit_sum += TF_legit[word]
        TF_spam_sum += TF_spam[word]

    for i, w in enumerate(wl):

        Pis = TF_spam[w]/(TF_spam_sum + M)
        Pih = TF_legit[w]/(TF_legit_sum + M)

        P_of_x_given_spam = P_of_x_given_spam * (np.power(Pis, data[i]))
        P_of_x_given_legit = P_of_x_given_legit * (np.power(Pih, data[i]))

    P_of_spam_given_x = (P_of_x_given_spam * P_of_spam)

    P_of_legit_given_x = (P_of_x_given_legit * P_of_legit)


    if P_of_spam_given_x / P_of_legit_given_x > 1:
        return 1
    else:
        return 0   


Generating feature vectors of size 10, 100 and 1000 with term frequencies on the test dataset.

In [38]:
test_data_10_mnb = gen_ft_mnb(10, processed_legit_docs_test) + gen_ft_mnb(10, processed_spam_docs_test)

test_data_100_mnb = gen_ft_mnb(100, processed_legit_docs_test) + gen_ft_mnb(100, processed_spam_docs_test)

test_data_1000_mnb = gen_ft_mnb(1000, processed_legit_docs_test) + gen_ft_mnb(1000, processed_spam_docs_test)

true_labels_test = [0] * 241 + [1] * 48

Classification on test set with N=10

In [39]:
pred_labels = []
for fv in test_data_10_mnb:
    pred_labels.append(multinomial_NB_TF(fv, top_10_IG_words))

true_labels_test = [0] * 241 + [1] * 48
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

[[236   5]
 [ 10  38]]


In [40]:
print_spam_precision_recall(cm)

Spam_precision = 79.16666666666666
Spam_recall = 88.37209302325581


Classification on test set with N=100

In [41]:
pred_labels = []
for fv in test_data_100_mnb:
    pred_labels.append(multinomial_NB_TF(fv, top_100_IG_words))

true_labels_test = [0] * 241 + [1] * 48
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

[[237   4]
 [  8  40]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


In [42]:
print_spam_precision_recall(cm)

Spam_precision = 83.33333333333334
Spam_recall = 90.9090909090909


Classification on test set with N=1000

In [43]:
pred_labels = []
for fv in test_data_1000_mnb:
    pred_labels.append(multinomial_NB_TF(fv, top_1000_IG_words))

true_labels_test = [0] * 241 + [1] * 48
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


[[240   1]
 [ 24  24]]


In [44]:
print_spam_precision_recall(cm)

Spam_precision = 50.0
Spam_recall = 96.0


#<h2>SVM with binary features</h2>
SVM was performed using both TF features and binary features seperately. The classifiers trained on binary features worked better. 
Parameters of SVM -  C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None.

In [45]:
train_data_10_nb_np = np.array(train_data_10_nb)
train_data_100_nb_np = np.array(train_data_100_nb)
train_data_1000_nb_np = np.array(train_data_1000_nb)

In [46]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [47]:
true_labels = [0] * 2170 + [1] * 432

SVM on feature vector of length 10

In [48]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_data_10_nb_np, true_labels)

test_data_10_nb = gen_ft_nb(10, processed_legit_docs_test) + gen_ft_nb(10, processed_spam_docs_test)
test_data_10_nb_np = np.array(test_data_10_nb)
test_pred = clf.predict(test_data_10_nb_np)

true_labels_test = [0] * 241 + [1] * 48
cm_test = confusion_matrix(true_labels_test, test_pred)
print(cm_test)
print_spam_precision_recall(cm_test)

[[238   3]
 [  4  44]]
Spam_precision = 91.66666666666666
Spam_recall = 93.61702127659575


SVM on feature vector of length 100

In [49]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_data_100_nb_np, true_labels)

test_data_100_nb = gen_ft_nb(100, processed_legit_docs_test) + gen_ft_nb(100, processed_spam_docs_test)
test_data_100_nb_np = np.array(test_data_100_nb)
test_pred = clf.predict(test_data_100_nb_np)
true_labels_test = [0] * 241 + [1] * 48
cm_test = confusion_matrix(true_labels_test, test_pred)
print(cm_test)
print_spam_precision_recall(cm_test)

[[241   0]
 [  2  46]]
Spam_precision = 95.83333333333334
Spam_recall = 100.0


SVM on feature vector of length 1000

In [50]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_data_1000_nb_np, true_labels)

test_data_1000_nb = gen_ft_nb(1000, processed_legit_docs_test) + gen_ft_nb(1000, processed_spam_docs_test)
test_data_1000_nb_np = np.array(test_data_1000_nb)
test_pred = clf.predict(test_data_1000_nb_np)
true_labels_test = [0] * 241 + [1] * 48
cm_test = confusion_matrix(true_labels_test, test_pred)
print(cm_test)
print_spam_precision_recall(cm_test)

[[241   0]
 [  0  48]]
Spam_precision = 100.0
Spam_recall = 100.0


#<h2>SVM with term frequency features</h2>

In [51]:
train_data_10_mnb_np = np.array(train_data_10_mnb)
train_data_100_mnb_np = np.array(train_data_100_mnb)
train_data_1000_mnb_np = np.array(train_data_1000_mnb)

SVM on feature vector of length 10

In [52]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_data_10_mnb_np, true_labels)

test_data_10_mnb = gen_ft_mnb(10, processed_legit_docs_test) + gen_ft_mnb(10, processed_spam_docs_test)
test_data_10_mnb_np = np.array(test_data_10_mnb)
test_pred = clf.predict(test_data_10_mnb_np)

true_labels_test = [0] * 241 + [1] * 48
cm_test = confusion_matrix(true_labels_test, test_pred)
print(cm_test)
print_spam_precision_recall(cm_test)

[[238   3]
 [ 12  36]]
Spam_precision = 75.0
Spam_recall = 92.3076923076923


SVM on feature vector of length 100



In [53]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_data_100_mnb_np, true_labels)

test_data_100_mnb = gen_ft_mnb(100, processed_legit_docs_test) + gen_ft_mnb(100, processed_spam_docs_test)
test_data_100_mnb_np = np.array(test_data_100_mnb)
test_pred = clf.predict(test_data_100_mnb_np)

true_labels_test = [0] * 241 + [1] * 48
cm_test = confusion_matrix(true_labels_test, test_pred)
print(cm_test)
print_spam_precision_recall(cm_test)

[[241   0]
 [ 12  36]]
Spam_precision = 75.0
Spam_recall = 100.0


SVM on feature vector of length 1000

In [54]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_data_1000_mnb_np, true_labels)

test_data_1000_mnb = gen_ft_mnb(1000, processed_legit_docs_test) + gen_ft_mnb(1000, processed_spam_docs_test)
test_data_1000_mnb_np = np.array(test_data_1000_mnb)
test_pred = clf.predict(test_data_1000_mnb_np)

true_labels_test = [0] * 241 + [1] * 48
cm_test = confusion_matrix(true_labels_test, test_pred)
print(cm_test)
print_spam_precision_recall(cm_test)

[[241   0]
 [  9  39]]
Spam_precision = 81.25
Spam_recall = 100.0


# Adversarial setting

This is an alternate approach for creating a classifier in an adversarial setting. This is a modification of whats is presented in the paper. I was not able to implement the exact techniques presented in the paper. So, I came up with an intuitive and simple approach to solve this problem. 

In this approch I iterate over the feature vector of every email. If a 0 is found it is converted to 1 and this modified feature vector is fed to the Bernoulli NB classifier. The classifier returns P(spam/x) and P(ham/x), store these values with their respective index at which the change was performed. Once this calculated revert the changes made at that index. This is repeated for all the indices of the feature vector. Now sort all probabilities in increasing order of P(spam/x). Since we are interested in generating emails that result in low values of P(spam/x), it makes sense to make changes in the order presented in this sorted list. 

For every email in the test set make change a 0 to a 1 at the index presented in the sorted list. After every change calculate P(spam/x) and P(ham/x). If P(spam/x) > P(ham/x) then break out of the loop i.e no more chnages are necessary for this email. If this condition does not hold true then continue making more changes at the selected indices. Keep a track of the cost for every change that is made. 

In [57]:
P_of_spam = (spam_count+1)/(spam_count + ham_count+2)
P_of_legit = (ham_count+1)/(spam_count + ham_count+2)

def binomial_NB_prob(data, wl):
    P_of_x_given_spam = 1
    P_of_x_given_legit = 1
    for i, w in enumerate(wl):
        try:
            if spam_docs_counts_d[w]:
                sp_word = spam_docs_counts_d[w] + 1           

        except KeyError:
            sp_word = 1    

        try:
            if legit_docs_counts_d[w]:
                lg_word = legit_docs_counts_d[w] + 1

        except KeyError:
            lg_word = 1       

        Pis = sp_word/(spam_count+2)
        Pih = lg_word/(ham_count+2)

        P_of_x_given_spam = P_of_x_given_spam * (np.power(Pis, data[i]) * np.power((1 - Pis),(1 - data[i])))
        P_of_x_given_legit = P_of_x_given_legit * (np.power(Pih, data[i]) * np.power((1 - Pih),(1 - data[i])))

    P_of_x = (P_of_spam * P_of_x_given_spam) + (P_of_legit * P_of_x_given_legit)
    P_of_spam_given_x = (P_of_x_given_spam * P_of_spam) / P_of_x
    P_of_legit_given_x = (P_of_x_given_legit * P_of_legit) / P_of_x


    return (P_of_spam_given_x, P_of_legit_given_x)  

In [58]:
test_data_10_nb = gen_ft_nb(10, processed_legit_docs_test) + gen_ft_nb(10, processed_spam_docs_test)

In [59]:
import copy 
original_probs = []
for te in test_data_10_nb:
    original_probs.append(binomial_NB_prob(te, top_10_IG_words))

In [60]:
modified_test_set = copy.deepcopy(test_data_10_nb)

In [61]:
new_probs_full = []
true_labels_test = [0] * 241 + [1] * 48

for roll, te in enumerate(test_data_10_nb):
    if true_labels_test[roll] == 1:
        new_prob_list = []
        for ind, val in enumerate(te):
            if val==0:
                modified_test_set[roll][ind] = 1
                P_of_spam_given_x, P_of_legit_given_x = binomial_NB_prob(modified_test_set[roll], top_10_IG_words)
                new_prob_list.append((P_of_spam_given_x, P_of_legit_given_x, ind))
                modified_test_set[roll][ind] = 0

            else:
                P_of_spam_given_x, P_of_legit_given_x = binomial_NB_prob(modified_test_set[roll], top_10_IG_words)
                new_prob_list.append((P_of_spam_given_x, P_of_legit_given_x, ind))

        new_probs_full.append(new_prob_list)

In [62]:
# sort probabilities
sorted_probs = []
for prob_tup_list in new_probs_full:
    sorted_probs.append(sorted(prob_tup_list, key=lambda tup: tup[0]))

In [63]:
mod_test_fin = copy.deepcopy(test_data_10_nb[-48:])

In [64]:
cost_per_email = []

for roll, te in enumerate(test_data_10_nb[-48:]):
    cost = 0
    if true_labels_test[-48:][roll] == 1:
        for ind, tup_list in enumerate(sorted_probs):
            for tup in tup_list:
                p1, p2, k = tup
                mod_test_fin[roll][k] = 1
                cost+=1
                P_of_spam_given_x, P_of_legit_given_x = binomial_NB_prob(mod_test_fin[roll], top_10_IG_words)

                if P_of_spam_given_x < P_of_legit_given_x:
                    flag = 1
                    break
                else:
                    continue
            if flag == 1:
                break
            else:
                continue                    
        if cost>3:
            mod_test_fin[roll] = test_data_10_nb[-48:][roll]
            cost = 0
            cost_per_email.append(cost)
        else:
            cost_per_email.append(cost)     

In [65]:
fin_test = test_data_10_nb[:241] + mod_test_fin
len(fin_test)

289

Performing classification on the emails modified by the adversary.

In [66]:
pred_labels = []
for fv in fin_test:
    pred_labels.append(binomial_NB(fv, top_10_IG_words))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)
print_spam_precision_recall(cm)

[[238   3]
 [ 42   6]]
Spam_precision = 12.5
Spam_recall = 66.66666666666666


Printing all cost values

In [67]:
cost_per_email

[1,
 0,
 1,
 0,
 2,
 2,
 0,
 1,
 3,
 1,
 1,
 2,
 2,
 0,
 3,
 3,
 3,
 1,
 1,
 2,
 3,
 3,
 3,
 1,
 0,
 1,
 0,
 3,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 3,
 1,
 3,
 3,
 2,
 2,
 1]

Minimum cost value

In [68]:
minimum_adversary_cost = sum(cost_per_email)/48
print(minimum_adversary_cost)

1.5833333333333333


Modified Classifier part

In [69]:
train_data_10_nb = gen_ft_nb(10, processed_legit_docs) + gen_ft_nb(10, processed_spam_docs)
modified_train_set = copy.deepcopy(train_data_10_nb)
true_labels = [0] * 2170 + [1] * 432

In [70]:
new_probs_full = []

for roll, te in enumerate(train_data_10_nb):
    if true_labels[roll] == 1:
        new_prob_list = []
        for ind, val in enumerate(te):
            if val==0:
                modified_train_set[roll][ind] = 1
                P_of_spam_given_x, P_of_legit_given_x = binomial_NB_prob(modified_train_set[roll], top_10_IG_words)
                new_prob_list.append((P_of_spam_given_x, P_of_legit_given_x, ind))
                modified_train_set[roll][ind] = 0

            else:
                P_of_spam_given_x, P_of_legit_given_x = binomial_NB_prob(modified_train_set[roll], top_10_IG_words)
                new_prob_list.append((P_of_spam_given_x, P_of_legit_given_x, ind))

        new_probs_full.append(new_prob_list)


In [71]:
# sort probabilities
sorted_probs = []
for prob_tup_list in new_probs_full:
    sorted_probs.append(sorted(prob_tup_list, key=lambda tup: tup[0]))

In [72]:
mod_train_fin = copy.deepcopy(train_data_10_nb[-432:])
train_orginal_probs = original_probs[-432:]

In [73]:
cost_per_email = []

for roll, te in enumerate(train_data_10_nb[-432:]):
    cost = 0
    if true_labels[-432:][roll] == 1:
        for ind, tup_list in enumerate(sorted_probs):
            for tup in tup_list:
                p1, p2, k = tup
                mod_train_fin[roll][k] = 1
                cost+=1
                P_of_spam_given_x, P_of_legit_given_x = binomial_NB_prob(mod_train_fin[roll], top_10_IG_words)

                if P_of_spam_given_x < P_of_legit_given_x:
                    flag = 1
                    break
                else:
                    continue
            if flag == 1:
                break
            else:
                continue            
        cost_per_email.append(cost)
        if cost==10:
            mod_train_fin[roll] = train_data_10_nb[-432:][roll]  

In [74]:
fin_train = train_data_10_nb[:2170] + mod_train_fin
len(fin_train)

2602

Modified classifier applied on the test data

In [75]:
fin_train_np = np.array(fin_train)
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(fin_train_np, true_labels)

test_data_10_nb = gen_ft_nb(10, processed_legit_docs_test) + gen_ft_nb(10, processed_spam_docs_test)
test_data_10_nb_np = np.array(test_data_10_nb)
test_pred = clf.predict(test_data_10_nb_np)

from sklearn.metrics import confusion_matrix
true_labels_test = [0] * 241 + [1] * 48
cm_test = confusion_matrix(true_labels_test, test_pred)
print(cm_test)
print_spam_precision_recall(cm_test)

[[239   2]
 [ 15  33]]
Spam_precision = 68.75
Spam_recall = 94.28571428571428


#(EXTRA) Adversarial setting 
This is an attempt to implement the exact technique that is presented in the paper. This approch did not give expected results hence I above modified approch was implemented instead of this. 

In [ ]:
P_of_spam = (spam_count+1)/(spam_count + ham_count+2)
P_of_legit = (ham_count+1)/(spam_count + ham_count+2)

def binomial_NB_prob(data, wl):
    P_of_x_given_spam = 1
    P_of_x_given_legit = 1
    for i, w in enumerate(wl):
        try:
            if spam_docs_counts_d[w]:
                sp_word = spam_docs_counts_d[w] + 1           

        except KeyError:
            sp_word = 1    

        try:
            if legit_docs_counts_d[w]:
                lg_word = legit_docs_counts_d[w] + 1

        except KeyError:
            lg_word = 1       

        Pis = sp_word/(spam_count+2)
        Pih = lg_word/(ham_count+2)

        P_of_x_given_spam = P_of_x_given_spam * (np.power(Pis, data[i]) * np.power((1 - Pis),(1 - data[i])))
        P_of_x_given_legit = P_of_x_given_legit * (np.power(Pih, data[i]) * np.power((1 - Pih),(1 - data[i])))

    P_of_x = (P_of_spam * P_of_x_given_spam) + (P_of_legit * P_of_x_given_legit)
    P_of_spam_given_x = (P_of_x_given_spam * P_of_spam) / P_of_x
    P_of_legit_given_x = (P_of_x_given_legit * P_of_legit) / P_of_x


    return (P_of_spam_given_x, P_of_legit_given_x)  

In [ ]:
def binomial_NB_prob_prod(data, wl):
    P_of_x_given_spam_prod = []
    P_of_x_given_legit_prod = []

    P_of_x_given_spam = 1
    P_of_x_given_legit = 1
    for i, w in enumerate(wl):
        try:
            if spam_docs_counts_d[w]:
                sp_word = spam_docs_counts_d[w] + 1           

        except KeyError:
            sp_word = 1    

        try:
            if legit_docs_counts_d[w]:
                lg_word = legit_docs_counts_d[w] + 1

        except KeyError:
            lg_word = 1       

        Pis = sp_word/(spam_count+2)
        Pih = lg_word/(ham_count+2)

        P_of_x_given_spam = P_of_x_given_spam * (np.power(Pis, data[i]) * np.power((1 - Pis),(1 - data[i])))
        P_of_x_given_spam_prod.append((np.power(Pis, data[i]) * np.power((1 - Pis),(1 - data[i]))))

        P_of_x_given_legit = P_of_x_given_legit * (np.power(Pih, data[i]) * np.power((1 - Pih),(1 - data[i])))
        P_of_x_given_legit_prod.append((np.power(Pih, data[i]) * np.power((1 - Pih),(1 - data[i]))))

    P_of_x = (P_of_spam * P_of_x_given_spam) + (P_of_legit * P_of_x_given_legit)

    P_of_spam_given_x = (P_of_x_given_spam * P_of_spam) / P_of_x

    P_of_legit_given_x = (P_of_x_given_legit * P_of_legit) / P_of_x


    return (P_of_x_given_spam_prod, P_of_x_given_legit_prod)

In [ ]:
def FindMCC(i, w, x, WL):
    if w<=0:
        return (0,[])

    if i==0:
        return (float('inf'), [])

    min_cost = float('inf')
    min_list = []

    for xi_prime in [1,0]:

        P_of_x_given_spam_prod, P_of_x_given_legit_prod = binomial_NB_prob_prod(x, WL)
        x_mod = x.copy()
        xi = x[i]
        x_mod[i] = xi_prime

        Mod_P_of_x_given_spam_prod, Mod_P_of_x_given_legit_prod = binomial_NB_prob_prod(x_mod, WL)

        LO_c_of_xi = np.log(P_of_x_given_spam_prod[i] / P_of_x_given_legit_prod[i])
        LO_c_of_xi_prime =  np.log(Mod_P_of_x_given_spam_prod[i] / Mod_P_of_x_given_legit_prod[i])

        if xi == xi_prime:
            delta_LO_i_xi_prime = 0
            Wi_of_xi_xi_prime = 0
        else:
            delta_LO_i_xi_prime = LO_c_of_xi - LO_c_of_xi_prime
            Wi_of_xi_xi_prime = 1

        if delta_LO_i_xi_prime >= 0:
            cur_cost, cur_list = FindMCC(i-1, w - delta_LO_i_xi_prime, x, WL)
            cur_cost = cur_cost + Wi_of_xi_xi_prime 
            cur_list.append((i, xi_prime))

            if cur_cost < min_cost:
                min_cost = cur_cost
                min_list = cur_list

    return (min_cost, min_list)                

In [ ]:
def A(x, word_list):

    Uc_pp = 1
    Uc_pn = -10
    Uc_np = -1
    Uc_nn = 1

    Ua_pp = 0
    Ua_pn = 0
    Ua_np = 20
    Ua_nn = 0

    P_of_spam_given_x, P_of_legit_given_x, P_of_x_given_spam, P_of_x_given_legit = binomial_NB_prob(x, word_list)

    LO_c_of_x = np.log(P_of_spam_given_x / P_of_legit_given_x)

    LT_of_Uc = np.log((Uc_nn - Uc_pn) / (Uc_pp - Uc_np))
    gap_of_x = LO_c_of_x - LT_of_Uc

    delta_U_A = Ua_np - Ua_pp

    n = 9
    W = gap_of_x
    min_cost, min_list = FindMCC(n, W, x, word_list)
    if binomial_NB(x, word_list) == 1 and min_cost < delta_U_A:
        return ['mod', (min_cost, min_list)]
    else:
        return x

In [ ]:
alterated_data = []
count = 0
for i, te in enumerate(test_data_10_nb):
    adv_out = A(te, top_10_IG_words)
    alterated_data.append(adv_out)   

In [ ]:
def modify(mod_list, true_list):
    nl = true_list.copy()
    for ind, val in mod_list:
        if val == 1:nl[ind] = val
        else:continue      
    return nl        

In [ ]:
print(alterated_data[0][1])

In [ ]:
final_altered_data = []
mod_count = 0
for mod_l, true_l in zip(alterated_data, test_data_10_nb):

    if mod_l[0] == 'mod':
        new_l = modify(mod_l[1][1], true_l)
        final_altered_data.append(new_l)
        print(new_l, true_l)
        if new_l == true_l:
            mod_count+=1

    else:
        final_altered_data.append(true_l)

Before adversarial modification

In [ ]:
pred_labels = []
for fv in test_data_10_nb:
    pred_labels.append(binomial_NB(fv, top_10_IG_words))

true_labels_test = [0] * 241 + [1] * 48
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)
print_spam_precision_recall(cm)

After adversarial modification

In [ ]:
pred_labels = []
for fv in final_altered_data:
    pred_labels.append(binomial_NB(fv, top_10_IG_words))

true_labels_test = [0] * 241 + [1] * 48
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_test, pred_labels)
print(cm)
print_spam_precision_recall(cm)